In [1]:
import numpy as np

In [2]:

class SimpleCNN:
    
    def __init__(self):
        self.layers = []
        self.pool_size = None
    
    def load_weights(self, weights):
        assert not self.layers, "Weights can only be loaded once"
        
        for k in range(weights.keys()):
            self.layers.append(weights['layer_{}'.format(k)])
    
    def predict(self, X):
        
        #CONV -> Relu -> Pool
        h = self.conv_layer(X,layer_i=0); X = h
        h = self.relu_layer(X); X = h
        h = self.pool_layer(X); X = h
        
        h = self.flatten_layer(X); X = h
        h = self.dense_layer(X, layer_i = 4); X = h
        h = self.softmax_layer(X); X = h
        max_i = self.classify(X)
        
        return max_i[0]
    
    def conv_layer(self, X, layer_i=0):
        
        features = self.layers[layer_i]["param0"]
        bias = self.layers[layer_i]["param1"]
        
        f = features[0].shape[-1]
        nb_features = features.shape[0]
        n = X.shape[2]
        image_channels = X.shape[1]
        nb_images = X.shape[0]
        conv_dim = n+f-1
        
        convolved_features = np.zeroes((nb_images,nb_features,conv_dim,conv_dim))
        
        for image_i in range(nb_images):
            for feature_i in range(nb_features):
                convolved_image = np.zeroes((conv_dim,conv_dim))
                for channel in range(image_channels):
                    feature = features[feature_i,channel,:, :]
                    image = X[image_i,channel,:,:]
                    convolved_image += self.conv2D(image,feature)
                    
                convolved_image += bias[feature_i]
                convolved_features[image_i,feature_i,:,:] = convolved_image
                
        return convolved_features
    
    @staticmethod
    def conv2D(image, feature):
        
        image_dim = np.array(image.shape)
        feature_dim = np.array(feature.shape)
        target_dim = image_dim + feature_dim -1
        
        fft_result = np.fft.fft2(image,target_dim)* np.fft.fft2(feature,target_dim)
        
        target = np.fft.fft2(fft_result).real
        
        return target
    
    def relu_layer(self, x):
        z = np.zeros_like(x)
        return np.where(x>z,x,z)
    
    #Pool Layer -Max Pool
    def pool_layer(self, convolved_features):
        nb_features = convolved_features.shape[0]
        nb_images = convolved_features.shape[1]
        conv_dim = convolved_features.shape[2]
        res_dim = int(conv_dim / self.pool_size)
        
        pooled_features = np.zeros((nb_features,nb_images,res_dim,res_dim))
        
        for image_i in range(nb_images):
            for feature_i in range(nb_features):
                for pool_row in range(res_dim):
                    row_start = pool_row * self.pool_size
                    row_end = row_start + self.pool_size
                    
                    for pool_col in range(res_dim):
                        col_start = pool_col * self.pool_size
                        col_end = col_start + self.pool_size
                        
                        patch = convolved_features[feature_i, image_i, row_start:row_end, col_start:col_end]
                
                    pooled_features[feature_i,image_i,pool_row,pool_col] = np.max(patch)
                
                return pooled_features
    
    def flatten_layer(self, X):
        flatX = np.zeros((X.shape[0],np.prod(X.shape[1:])))
        for i in range(X.shape[0]):
            flatX[i,:] = X[i].flatten(order='C')
        return flatX
    
    #FC layer
    def dense_layer(self,X, layer_i=0):
        W = self.layers[layer_i]["param0"]
        b = self.layers[layer_i]["param1"]
        output = np.dot(X,W) + b
        return output
    
    def softmax_layer(self, w):
        maxes = np.amax(w, axis=1)
        maxes = maxes.reshape(maxes.shape[0],1)
        e = np.exp(w - maxes)
        dist = e / np.sum(e, axis=1, keepdims=True)
        return dist
    
    def classify(self, X):
        return X.argmax(axis=-1)
        